<a href="https://colab.research.google.com/github/chabryl/AI-Redirect-Mapping/blob/main/Aktuell_redirect_mapping_qdrant_live.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers
!pip install qdrant-client
from sentence_transformers import SentenceTransformer
import pandas as pd
import io
import time
from google.colab import files
from qdrant_client import QdrantClient
from qdrant_client.http import models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully u

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Initialisierung des SentenceTransformer-Modells

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def encode_texts(df, selected_columns):
    """
    Kodiert die Texte im gegebenen DataFrame mit dem SentenceTransformer-Modell.
    """
    df['combined_text'] = df[selected_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    embeddings = model.encode(df['combined_text'].tolist(), show_progress_bar=True)
    return embeddings

    start_time = time.time()
    embeddings = model.encode(df['combined_text'].tolist(), show_progress_bar=True)
    end_time = time.time()

    print(f"Kodierungszeit: {end_time - start_time} Sekunden")



def detect_delimiter(file_content):
    """
    Versucht, das Trennzeichen in der CSV-Datei zu erkennen.
    """
    file_str = file_content.getvalue().decode('utf-8')
    first_line = file_str.splitlines()[0]

    if ',' in first_line:
        return ','
    elif ';' in first_line:
        return ';'
    else:
        raise ValueError("Unbekanntes Trennzeichen in der CSV-Datei.")

def upload_and_read_csv(file_name):
    """
    Lädt eine CSV-Datei hoch und versucht sie zu lesen.
    """
    print(f"Bitte laden Sie die {file_name} Datei hoch.")
    uploaded = files.upload()
    file_content = io.BytesIO(uploaded[list(uploaded.keys())[0]])

    delimiter = detect_delimiter(file_content)
    file_content.seek(0)

    try:
        df = pd.read_csv(file_content, sep=delimiter, skip_blank_lines=True, on_bad_lines='skip')
        print(f"CSV-Datei erfolgreich eingelesen mit Trennzeichen: '{delimiter}'")
    except pd.errors.ParserError:
        print(f"Fehler beim Einlesen der Datei {file_name}. Bitte überprüfe das Dateiformat.")
        return None

    print("Spaltennamen in der Datei:", df.columns)
    print("Erste Zeilen der Datei:")
    print(df.head())

    return df

def upsert_to_qdrant(client, collection_name, embeddings, ids):
    """
    Fügt Embeddings in die Qdrant-Kollektion ein.
    """
    batch_size = 100
    for i in range(0, len(embeddings), batch_size):
        batch_embeddings = embeddings[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]
        points = [
            models.PointStruct(
                id=i + 1, # Use a simple integer ID. Increment for each point.
                vector=embedding.tolist(),
                payload={"url": id} # Store the URL in the payload instead
            ) for i, (id, embedding) in enumerate(zip(batch_ids, batch_embeddings))
        ]
        client.upsert(collection_name=collection_name, points=points)

def query_qdrant(client, collection_name, query_embeddings, top_k=1):
    """
    Führt eine Abfrage auf der Qdrant-Kollektion durch.
    """
    results = []
    for query in query_embeddings:
        search_result = client.search(
            collection_name=collection_name,
            query_vector=query.tolist(),
            limit=top_k
        )
        results.append(search_result)
    return results

def main():
    print("Willkommen zum KI-gestützten Redirect-Mapping-Tool!")

    # Datei-Upload
    origin_df = upload_and_read_csv("origin.csv")
    destination_df = upload_and_read_csv("destination.csv")

    if origin_df is None or destination_df is None:
        print("Fehler beim Einlesen der Dateien. Beenden des Programms.")
        return

    # Überprüfen, ob die Spalte 'url' existiert
    if 'url' not in origin_df.columns or 'url' not in destination_df.columns:
        print("Spalte 'url' nicht in den Daten gefunden. Bitte prüfe die Spaltennamen.")
        return

    print("Origin DataFrame columns:", origin_df.columns)
    print("Destination DataFrame columns:", destination_df.columns)

    common_columns = list(set(origin_df.columns) & set(destination_df.columns))
    print("Verfügbare Spalten:", common_columns)

    while True:
       selected_columns_input = input("Geben Sie die zu verwendenden Spalten ein (durch Komma getrennt): ").split(',')
       selected_columns = [col.strip() for col in selected_columns_input if col.strip() != 'language']

       if all(col in origin_df.columns and col in destination_df.columns for col in selected_columns):
           break
       else:
           print("Ungültige Spaltenauswahl. Bitte wählen Sie nur Spalten aus, die in beiden DataFrames vorhanden sind.")
           print("Verfügbare Spalten:", common_columns)

    # Kodierung der Texte
    print("Kodiere Ursprungs-Texte...")
    origin_embeddings = encode_texts(origin_df, selected_columns)
    print("Kodiere Ziel-Texte...")
    destination_embeddings = encode_texts(destination_df, selected_columns)

    print("Initialisiere Qdrant...")
    client = QdrantClient(
        url="https://b80e8e91-bbbe-4a9b-9b36-d2ab69344ba1.europe-west3-0.gcp.cloud.qdrant.io",
        api_key="Upi_Hx819u4hFYNKaUH5np9BNFFHd44Vsey3qS_WI_tHGjGnIP4ULw"  # Ersetzen Sie dies durch Ihren tatsächlichen API-Key
    )
    collection_name = "url-mapping"

    # Erstellen der Kollektion (falls sie noch nicht existiert)
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=len(destination_embeddings[0]), distance=models.Distance.COSINE),
    )

    # Einfügen der Ziel-Embeddings in Qdrant
    print("Füge Ziel-Embeddings in Qdrant ein...")
    upsert_to_qdrant(client, collection_name, destination_embeddings, destination_df['url'].tolist())

    # Abfrage der Qdrant-Datenbank
    print("Führe Ähnlichkeitssuche durch...")
    results = query_qdrant(client, collection_name, origin_embeddings)

    # Verarbeitung der Ergebnisse
    matched_urls = []
    similarity_scores = []
    for result in results:
        if result:
            matched_urls.append(result[0].payload['url'])
            similarity_scores.append(result[0].score)
        else:
            matched_urls.append(None)
            similarity_scores.append(0)

    # Erstellung des Ergebnisberichts
    report = pd.DataFrame({
        'origin_url': origin_df['url'],
        'matched_url': matched_urls,
        'similarity_score': similarity_scores,
    })


    # Anzeige der Ergebnisse
    print(report.head())

    # Speichern der Ergebnisse als CSV
    report.to_csv('redirect_mapping_results.csv')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Kodiere Ziel-Texte...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Initialisiere Qdrant...


<ipython-input-2-59075e85908c>:136: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Füge Ziel-Embeddings in Qdrant ein...
Führe Ähnlichkeitssuche durch...
                                          origin_url  \
0            https://www.neumeisterhydraulik.de/home   
1  https://www.neumeisterhydraulik.de/home/doppel...   
2  https://www.neumeisterhydraulik.de/home/servic...   
3  https://www.neumeisterhydraulik.de/home/servic...   
4  https://www.neumeisterhydraulik.de/home/untern...   

                                         matched_url  similarity_score  
0  https://neumeisterhydraulik.hotbytes.rocks/hom...          0.783967  
1  https://neumeisterhydraulik.hotbytes.rocks/en/...          0.832767  
2  https://neumeisterhydraulik.hotbytes.rocks/hom...          0.892626  
3  https://neumeisterhydraulik.hotbytes.rocks/hom...          0.817992  
4  https://neumeisterhydraulik.hotbytes.rocks/hom...          0.953031  
